In [ ]:
import pickle
from pathlib import Path

import yaml
import pandas as pd

config_path = Path("./config.yaml")
with config_path.open("r") as f:
    config = yaml.safe_load(f)

base_path = Path(config["base_path"])
stats_path = base_path / config["batches"][0] / config["experiments"][0] / config["trials"][0] / "logs" / "training_stats_checkpoint.pkl"

with stats_path.open("rb") as f:
    training_stats = pickle.load(f)

print(training_stats.keys())
df = pd.DataFrame(training_stats)
df

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(df["reward"])
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Training Reward")
plt.grid(True)
plt.show()